# Use Langchain to interact with a SQL database 

The following code showcases an example of the Databricks SQL Agent. With the Databricks SQL agent any Databricks users can interact with a specified schema in Databrick Unity Catalog and generate insights on their data.

## Requirements

- To use this notebook, please provide your OpenAI API Token.
- Databricks Runtime 13.3 ML and above

**SOURCE: https://docs.databricks.com/en/large-language-models/langchain.html**  

The example mentioned in the notebook has outdated packages. 

### Imports and checking of versions of libraries and environment

Databricks recommends the latest version of `langchain` and the `databricks-sql-connector`.

In [0]:
%pip install --upgrade openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/99/ae/e8fb328fc0fc20ae935950b1f7160de8e2631a5997c2398c9b8a8cc502f8/openai-1.44.0-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/47/e7/5d88031cd743c62199b125181a591b1671df3ff2f6e102df85c58d8f7d31/jiter-0.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.11 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/367.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/319

In [0]:
%pip install --upgrade sqlalchemy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/b0/33/0806c5fc85bc022b6250313a01e4e504a1f5c12fe5e48ab52d0b4c2c0f81/SQLAlchemy-2.0.34-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.2 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 1.4/3.2 MB 43.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Not uninstalling sqlalchemy at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-abe5c620-a11f-44f2-b20e-9698491f7a82
    Can't uninstall 'SQLAlchemy'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or d

In [0]:
%pip install --upgrade langchain databricks-sql-connector langchain-openai


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/0d/29/635343c0d155997569b544d26da5a2a9ebade2423baffc9cd6066b01a386/langchain-0.2.16-py3-none-any.whl.metadata
  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for databricks-sql-connector from https://files.pythonhosted.org/packages/9c/b1/0a4b9f376717d28b30a1d3d5de4dfc94155e8c604c9c73a6e277a6cc64d1/databricks_sql_connector-3.4.0-py3-none-any.whl.metadata
  Using cached databricks_sql_connector-3.4.0-py3-none-any.whl.metadata (4.4 kB)
  Obtaining dependency information for langchain-openai from https://files.pythonhosted.org/packages/0e/f1/12cd80e136b6bf5f498bee178084e71d48f664e5639e9cc7f900244d9f1c/langchain_openai-0.1.23-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3.0,>=0.2.38 from 

In [0]:
!pip show openai

Name: openai
Version: 1.44.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-abe5c620-a11f-44f2-b20e-9698491f7a82/lib/python3.11/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai


In [0]:
!pip show langchain

Name: langchain
Version: 0.2.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-abe5c620-a11f-44f2-b20e-9698491f7a82/lib/python3.11/site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [0]:
!pip show databricks-sql-connector

Name: databricks-sql-connector
Version: 3.4.0
Summary: Databricks SQL Connector for Python
Home-page: 
Author: Databricks
Author-email: databricks-sql-connector-maintainers@databricks.com
License: Apache-2.0
Location: /local_disk0/.ephemeral_nfs/envs/pythonEnv-abe5c620-a11f-44f2-b20e-9698491f7a82/lib/python3.11/site-packages
Requires: lz4, numpy, oauthlib, openpyxl, pandas, pyarrow, requests, thrift, urllib3
Required-by: 


In [0]:
import sys  
print("Python version:", sys.version)  

Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [0]:
# Display the cluster name
print("Cluster Name:", spark.conf.get("spark.databricks.clusterUsageTags.clusterName"))

# Display the Databricks Runtime version of the cluster
print("Cluster Version:", spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion"))

Cluster Name: Amaan S's Cluster
Cluster Version: 15.3.x-cpu-ml-scala2.12


In [0]:
dbutils.library.restartPython()

In [0]:
import os
os.environ["OPENAI_API_KEY"] = ""

### SQL Database Agent

This is an example of how to interact with a certain schema in Unity Catalog. Please note that the agent can't create new tables or delete tables. It can only query tables.

The database instance is created within:
```
db = SQLDatabase.from_databricks(catalog="...", schema="...")
```
And the agent (and the required tools) are created by:
```
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, **kwargs)
```

## Using Open AI

In [0]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI

db = SQLDatabase.from_databricks(catalog="samples", schema="nyctaxi")
llm = OpenAI(temperature=.7)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [0]:
agent.invoke("What is the longest trip distance and how long did it take?")



> Entering new SQL Agent Executor chain...
 We need to find the longest trip distance and its duration.
Action: sql_db_query
Action Input: SELECT MAX(trip_distance), duration FROM tripsError: (databricks.sql.exc.ServerOperationError) [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `duration` cannot be resolved. Did you mean one of the following? [`fare_amount`, `pickup_zip`, `dropoff_zip`, `trip_distance`, `tpep_pickup_datetime`]. SQLSTATE: 42703; line 1 pos 27
[SQL: SELECT MAX(trip_distance), duration FROM trips]
(Background on this error at: https://sqlalche.me/e/20/4xp6)There is no 'duration' column in the 'trips' table.
Action: sql_db_schema
Action Input: trips
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_dropoff_datetime TIMESTAMP, 
	trip_distance FLOAT, 
	fare_amount FLOAT, 
	pickup_zip INT, 
	dropoff_zip INT
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'enabled')

/*
3 rows from trips table:
tpep_pick

{'input': 'What is the longest trip distance and how long did it take?',
 'output': 'The longest trip distance is 30.6 and it took 43 minutes and 31 seconds.'}

In [0]:
agent.invoke("find out the zipcode of the trip with the highest fare")



> Entering new SQL Agent Executor chain...
 I should use the sql_db_query checker to make sure my query is correct
Action: sql_db_query_checker
Action Input: SELECT zipcode FROM trips ORDER BY fare DESC LIMIT 1
SELECT zipcode 
FROM trips 
WHERE zipcode NOT IN (NULL) 
ORDER BY fare DESC 
LIMIT 1 I should use the sql_db_query tool to execute my query and get the result
Action: sql_db_query
Action Input: SELECT zipcode FROM trips ORDER BY fare DESC LIMIT 1Error: (databricks.sql.exc.ServerOperationError) [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `zipcode` cannot be resolved. Did you mean one of the following? [`pickup_zip`, `dropoff_zip`, `fare_amount`, `trip_distance`, `tpep_pickup_datetime`]. SQLSTATE: 42703; line 1 pos 7
[SQL: SELECT zipcode FROM trips ORDER BY fare DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/4xp6) I should use the sql_db_schema tool to check which table has the correct column name
Action: sql_db_s

{'input': 'find out the zipcode of the trip with the highest fare',
 'output': '10013'}

## Using Azure Open AI

In [0]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import OpenAI, AzureChatOpenAI

db = SQLDatabase.from_databricks(catalog="samples", schema="nyctaxi")

llm = AzureChatOpenAI(
    temperature=0.7,
    api_key="",
    api_version = "2024-02-01",
    azure_endpoint="",
    model="gpt4o"
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [0]:
agent.invoke("What is the longest trip distance and how long did it take?")



> Entering new SQL Agent Executor chain...
To answer this question, I need to determine which tables are present in the database and identify the one that contains information about trips, their distances, and durations.

Action: List all tables in the database.
Action Input: ""List all tables in the database. is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].I should use the correct tool to list all tables in the database.

Action: sql_db_list_tables
Action Input: ""tripsThe database contains a table named "trips." I need to inspect the schema of the "trips" table to determine which columns are available, particularly those related to trip distances and durations.

Action: sql_db_schema
Action Input: trips
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_dropoff_datetime TIMESTAMP, 
	trip_distance FLOAT, 
	fare_amount FLOAT, 
	pickup_zip INT, 
	dropoff_zip INT
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefa

{'input': 'What is the longest trip distance and how long did it take?',
 'output': 'The longest trip distance is 30.6 units, and it took 43 minutes and 31 seconds.'}

In [0]:
agent.invoke("find out the zipcode of the trip with the highest fare")



> Entering new SQL Agent Executor chain...
To find the zipcode of the trip with the highest fare, I need to:
1. List all the tables in the database to identify where trip information is stored.
2. Identify the schema of the relevant table to find the fields related to fare and zipcode.
3. Write and check an SQL query to find the trip with the highest fare and retrieve its associated zipcode.

Let's start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: ""tripsThe database contains a table named "trips". Next, I need to examine the schema of the "trips" table to identify the fields related to fare and zipcode.

Action: sql_db_schema
Action Input: "trips"
CREATE TABLE trips (
	tpep_pickup_datetime TIMESTAMP, 
	tpep_dropoff_datetime TIMESTAMP, 
	trip_distance FLOAT, 
	fare_amount FLOAT, 
	pickup_zip INT, 
	dropoff_zip INT
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'enabled')

/*
3 rows from trips table:
tpep_pickup_datetime	tpep

{'input': 'find out the zipcode of the trip with the highest fare',
 'output': 'The zipcode of the trip with the highest fare is `10013` for pickup and `7008` for dropoff.'}